# Group conversion factors for all outputs into relevant gases

In [4]:
# Import packages

import numpy as np
import pandas as pd

In [5]:
# Define paths
input_path = '../data/'
ecoInvent_files = [input_path+'extracted/conversionFactors_from_ecoinvent_IPCC2013_w_uncertainties.csv']
carbonMinds_files = [input_path+'extracted/conversionFactors_from_CarbonMinds_IPCC2013_w_uncertainties.csv']

output_path = '../data/processed/'

In [16]:
# EcoInvent + CarbonMinds -> Emissions
def import_emissions(files, source_name, region_names=False):
    emissions = pd.DataFrame()
    for file in files:
        emissions = pd.concat([emissions, pd.read_csv(file)])

    emissions = emissions.drop(columns=['Unnamed: 0']).rename(columns={'name': 'Source', 'CO2e': 'Cradle-to-gate'})
    emissions['Provenance'] = source_name

    if region_names is not False:
        # Get relevant values for region
        located = pd.DataFrame()
        for code in region_names:
            located = pd.concat([located, emissions[emissions['location']==code]])
        return located.groupby(['Source']).first().reset_index()

    else: return emissions

regions = ['GLO', 'GB', 'RER', 'RoW']
ei_emissions = import_emissions(ecoInvent_files, 'EcoInvent 3.8', regions)
cm_emissions = import_emissions(carbonMinds_files, 'Carbon Minds', regions)

# Groupings
emission_types = ['CO2e_20a', 'CO2e_100a', 'Carbon dioxide', 'Carbon monoxide', 'Chloroform', 'Dinitrogen monoxide', 'Ethane', 'Methane', 'Nitric oxide', 'Nitrogen fluoride', 'Perfluoropentane', 'Sulfur hexafluoride', 'Other']
emission_groups = [['CO2e_20a'], ['CO2e_100a'], ['Carbon dioxide, fossil', 'Carbon dioxide, from soil or biomass stock', 'Carbon dioxide, land transformation', 'Carbon dioxide, to soil or biomass stock'], ['Carbon monoxide, biogenic', 'Carbon monoxide, fossil', 'Carbon monoxide, from soil or biomass stock', 'Carbon monoxide, non-fossil'], ['Chloroform'], ['Dinitrogen monoxide'], ['Ethane, 1,1,1,2-tetrafluoro-, HFC-134a', 'Ethane, 1,1,1-trichloro-, HCFC-140', 'Ethane, 1,1,1-trifluoro-, HFC-143a', 'Ethane, 1,1,2-trichloro-1,2,2-trifluoro-, CFC-113', 'Ethane, 1,1-dichloro-1-fluoro-, HCFC-141b', 'Ethane, 1,1-difluoro-, HFC-152a', 'Ethane, 1,2-dichloro-', 'Ethane, 1,2-dichloro-1,1,2,2-tetrafluoro-, CFC-114', 'Ethane, 1-chloro-1,1-difluoro-, HCFC-142b', 'Ethane, 2,2-dichloro-1,1,1-trifluoro-, HCFC-123',  'Ethane, 2-chloro-1,1,1,2-tetrafluoro-, HCFC-124', 'Ethane, chloropentafluoro-, CFC-115', 'Ethane, hexafluoro-, HFC-116', 'Ethane, pentafluoro-, HFC-125'], ['Methane', 'Methane, biogenic', 'Methane, bromo-, Halon 1001', 'Methane, bromochlorodifluoro-, Halon 1211', 'Methane, bromotrifluoro-, Halon 1301','Methane, chlorodifluoro-, HCFC-22', 'Methane, chlorotrifluoro-, CFC-13', 'Methane, dichloro-, HCC-30', 'Methane, dichlorodifluoro-, CFC-12', 'Methane, dichlorofluoro-, HCFC-21', 'Methane, difluoro-, HFC-32', 'Methane, fossil', 'Methane, from soil or biomass stock', 'Methane, monochloro-, R-40', 'Methane, non-fossil', 'Methane, tetrachloro-, CFC-10', 'Methane, tetrachloro-, R-10', 'Methane, tetrafluoro-, R-14', 'Methane, trichlorofluoro-, CFC-11', 'Methane, trifluoro-, HFC-23'], ['Nitric oxide'], ['Nitrogen fluoride'], ['Perfluoropentane'], ['Sulfur hexafluoride'], ['VOC, volatile organic compounds, unspecified origin']]

for type, cols in zip(emission_types, emission_groups):
    ei_emissions[type] = np.sum(ei_emissions[cols], axis=1)
    ei_emissions[type+'_sigma'] = np.sum(ei_emissions[[col+'_sigma' for col in cols]], axis=1)
    cm_emissions[type] = np.sum(cm_emissions[cols], axis=1)
    cm_emissions[type+'_sigma'] = np.sum(cm_emissions[[col+'_sigma' for col in cols]], axis=1)
ei_emissions = ei_emissions[list(ei_emissions.columns[:3])+emission_types+[i+'_sigma' for i in emission_types]]
cm_emissions = cm_emissions[list(cm_emissions.columns[:3])+emission_types+[i+'_sigma' for i in emission_types]]

cm_emissions.head()

,Source,generalComment,location,CO2e_20a,CO2e_100a,Carbon dioxide,Carbon monoxide,Chloroform,Dinitrogen monoxide,Ethane,...,Carbon monoxide_sigma,Chloroform_sigma,Dinitrogen monoxide_sigma,Ethane_sigma,Methane_sigma,Nitric oxide_sigma,Nitrogen fluoride_sigma,Perfluoropentane_sigma,Sulfur hexafluoride_sigma,Other_sigma
0,"1,4-butanediol; consumption mix",The dataset represents a cradle-to-gate invent...,GLO,11.122994,8.864773,7.406913,0.150288,2.105922e-08,0.000131,1.417193e-07,...,0.015029,2.105922e-09,0.000013,1.417193e-08,0.002702,0.0,2.516730e-14,0.0,4.135234e-08,0.0
1,"1,4-butanediol; production mix",The dataset represents a cradle-to-gate invent...,GLO,11.292846,8.978761,7.482141,0.156374,2.145322e-08,0.000133,1.404861e-07,...,0.015637,2.145322e-09,0.000013,1.404861e-08,0.002747,0.0,2.560910e-14,0.0,4.202612e-08,0.0
2,"1,4-butanediol; technology mix; Reppe process ...",The dataset represents a cradle-to-gate invent...,GLO,8.655912,7.323702,6.567323,0.002942,1.638106e-08,0.000088,2.222268e-07,...,0.000294,1.638106e-09,0.000009,2.222268e-08,0.002401,0.0,2.148632e-14,0.0,3.307359e-08,0.0
3,2-ethylhexanol; consumption mix,The dataset represents a cradle-to-gate invent...,GLO,3.412158,2.920078,2.637113,0.001266,4.789950e-09,0.000047,3.670963e-08,...,0.000127,4.789950e-10,0.000005,3.670963e-09,0.000885,0.0,5.387338e-15,0.0,9.261296e-09,0.0
4,2-ethylhexanol; production mix,The dataset represents a cradle-to-gate invent...,GLO,3.579166,3.035431,2.723344,0.001262,5.051024e-09,0.000051,3.170732e-08,...,0.000126,5.051024e-10,0.000005,3.170732e-09,0.000979,0.0,4.739055e-15,0.0,9.493586e-09,0.0


In [18]:
# Output to file
ei_emissions.to_csv(output_path+'conversionFactors_ecoinvent_grouped.csv')
cm_emissions.to_csv(output_path+'conversionFactors_carbonMinds_grouped.csv')